In [ ]:
### Select only the roads from classification result ###
### Fill small holes in road polygons ###

import fiona
from shapely.geometry import shape, Polygon

# Open classification result; add path to '.shp'
with fiona.collection('.shp', 'r') as infile:    
    polygons = (elem for elem in infile)
    meta = infile.meta
    # Create output shapefile; add path to '.shp'
    with fiona.collection('.shp', 'w', **meta) as dst:
        c = 0
        for poly in polygons:
            c = c + 1
            # Adapt according to attribute name and attribute that describes the road class
            if poly['properties']['predicted'] == 2:
                myroad = shape(poly['geometry'])
                newcoords = []
                for coord in poly['geometry']['coordinates']:
                    # Choose only inner polygons larger than 500m²
                    if Polygon(coord).area >= 500:
                        newcoords.append(coord)                
                res = {}
                res['properties'] = poly['properties']
                res['geometry'] = poly['geometry']
                res['geometry']['coordinates'] = newcoords
                dst.write(res)
                print('added polygon ' + str(c) + ' of ' + str(len(infile)))